In [4]:
import gymnasium as gym 
from gymnasium import Env
from gymnasium.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete

import numpy as np 
import random 
import os

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

In [13]:
Discrete(3).sample()

np.int64(0)

In [40]:
Box(0,1, shape=(3,3)).sample()

array([[0.84923726, 0.21640626, 0.7197457 ],
       [0.5919928 , 0.72793496, 0.8996949 ],
       [0.67918146, 0.4627312 , 0.05154022]], dtype=float32)

In [41]:
Tuple((Discrete(3),Box(0,1,shape=(3,)),MultiBinary(4))).sample()

(np.int64(0),
 array([0.39772698, 0.5619476 , 0.916111  ], dtype=float32),
 array([0, 1, 1, 1], dtype=int8))

In [42]:
Dict({'heigth':Discrete(2),"speed":Box(0,100, shape=(1,)),"color":MultiBinary(4)}).sample()

{'color': array([0, 1, 0, 1], dtype=int8),
 'heigth': np.int64(0),
 'speed': array([99.5864], dtype=float32)}

In [27]:
MultiBinary(4).sample()

array([1, 0, 0, 1], dtype=int8)

In [47]:
MultiDiscrete([5,2,2,5]).sample()

array([4, 0, 0, 4])

In [96]:
class ShowerEnv(Env):
    def __init__(self):
        self.action_space = Discrete(3)
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        self.state = 38 + random.randint(-3, 3)
        self.shower_length = 60

    def step(self, action):
        # apply temp adj
        self.state += action - 1

        # Decrease shower time
        self.shower_length -= 1

        # calculate reward
        if 37 <= self.state <= 39:
            reward = 1
        else:
            reward = -1

        # ✅ changed: replaced `done` with `terminated` and `truncated`
        terminated = self.shower_length <= 0
        truncated = False  # not using truncation

        info = {}

        # ✅ changed: return 5 values
        return np.array([self.state], dtype=np.float32), reward, terminated, truncated, info

    def render(self):
        pass

    def reset(self, *, seed=None, options=None):
        super().reset(seed=seed)
        self.state = np.array([38 + random.randint(-3, 3)], dtype=np.float32)
        self.shower_length = 60
        return self.state, {}


In [97]:
env = ShowerEnv()

In [98]:
env.observation_space.sample()

array([53.551777], dtype=float32)

In [99]:
env.action_space.sample()

np.int64(0)

In [103]:
episodes = 5
for episode in range(1, episodes + 1):
    obs, _ = env.reset()
    terminated = False
    truncated = False
    score = 0

    while not (terminated or truncated):
        env.render()
        action = env.action_space.sample()
        obs, reward, terminated, truncated, info = env.step(action)
        score += reward

    print('Episode:{} Score:{}'.format(episode, score))

env.close()


Episode:1 Score:-60
Episode:2 Score:-54
Episode:3 Score:-52
Episode:4 Score:-60
Episode:5 Score:34


In [101]:
log_path = os.path.join('Training','Logs')
model = PPO('MlpPolicy',env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [105]:
model.learn(total_timesteps=20000)

Logging to Training/Logs/PPO_22
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -16.7    |
| time/              |          |
|    fps             | 2663     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60           |
|    ep_rew_mean          | -18.5        |
| time/                   |              |
|    fps                  | 1903         |
|    iterations           | 2            |
|    time_elapsed         | 2            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0030421512 |
|    clip_fraction        | 0.0274       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.551       |
|    explained_variance   

In [106]:
shower_path = os.path.join('Training','Saved Models','Shower_model_ppo')

In [107]:
model.save(shower_path)

In [108]:
del model

In [109]:
model = PPO.load(shower_path,env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [119]:
evaluate_policy(model, env, n_eval_episodes=10,render=True)

(np.float64(12.0), np.float64(58.787753826796276))